In [5]:
import DataPipeline
#import model
import pandas as pd
from sklearn.impute import KNNImputer

In [16]:
df = pd.read_csv("../data/Kagg_life_data.csv")

df_floats = df.drop(columns=["Country", "Status"])
print(df_floats.isna().sum())

imputer = KNNImputer().fit(df_floats)

imputed_df = imputer.transform(df_floats)

df[df_floats.columns] = imputed_df
print(df.isna().sum())

Year                                 0
Life expectancy                      0
Adult Mortality                      0
infant deaths                        0
Alcohol                            188
percentage expenditure               0
Hepatitis B                        536
Measles                              0
 BMI                                32
under-five deaths                    0
Polio                               19
Total expenditure                  221
Diphtheria                          19
 HIV/AIDS                            0
GDP                                411
Population                         612
 thinness  1-19 years               32
 thinness 5-9 years                 32
Income composition of resources    144
Schooling                          144
dtype: int64
Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths         